# Question 1 section:

Do some necessary imports:

In [2]:
import numpy as np
import pandas as pd
#Further necessary imports:
#!pip install geopy
#from geopy import geocoder # convert an address into latitude and longitude values


Get started with reading the table:

In [3]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data_raw = pd.read_html(URL)

Transfer the data list to the processing dataframe and cleanse it:

In [4]:
df_toronto_postal = data_raw[0]
#Drop entries with empty borough:
df_toronto_postal.drop(df_toronto_postal[df_toronto_postal.Borough=='Not assigned'].index,inplace=True)
df_toronto_postal.reset_index(drop=True,inplace=True)
#Check for duplicate Postal Codes to combine their neighbourhoods:
amount_duplicates = df_toronto_postal.duplicated(subset='Postal Code', keep=False).value_counts()
amount_duplicates

False    103
dtype: int64

No duplicate postal codes present, therefore no neighborhood merge needed

In [5]:
for index, row in df_toronto_postal.iterrows():
    input_neighborhood = row['Neighbourhood']
    if input_neighborhood == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
#Print shape:
df_toronto_postal.shape
#df_toronto_postal.head()

(103, 3)

### ________________________________________________________

# Question 2 section:

Start of section for question 2:

In [67]:
Neglected the following code due to issues with Geocoder:

#Fetch existing table and expand it with columns latitude and longitude:
#df_toronto = df_toronto_postal
#df_toronto["Latitude"] = ""
#df_toronto["Longitude"] = ""
#df_toronto.head()

##Use of the provided code to fetch the coordinates:

#import geocoder # import geocoder
#!pip3 install geopy
#import geocoder
#from geopy.geocoders import Nominatim

#initialize your variable to None
#lat_lng_coords = None

#insert code to loop through the series of postal codes:
#for index, row in df_toronto_postal.iterrows():
#    input_postal_code = row['Postal Code']
#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(input_postal_code))
#        lat_lng_coords = g.latlng

#    row['Latitude'] = lat_lng_coords[0]
#    row['longitude'] = lat_lng_coords[1]
    

SyntaxError: invalid syntax (<ipython-input-67-b6de92ce7206>, line 1)

Import Data from provided csv:

In [6]:
URL_csv='https://cocl.us/Geospatial_data'
coordinates_data=pd.read_csv(URL_csv)
coordinates_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge downloaded data with existing data:

In [7]:
df_toronto = df_toronto_postal.merge(coordinates_data,how='inner',on='Postal Code')
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Question 3 section:

In [8]:
#Start with the existing Dataframe and, as instructed, drop boroughs without 'Toronto' in their name:
df_toronto_nb = df_toronto[df_toronto.Borough.str.contains('Toronto')]
df_toronto_nb.reset_index(drop=True,inplace=True)
df_toronto_nb.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [9]:
#Install and import Folium

!conda config --set channel_priority true

!conda config --set channel_priority false

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.12.5          |   py37h89c1867_0         143 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    widgetsnbextension-3.5.1   |   py37h89c1867_4         1.8 MB  conda-forge
    ------------------------------------------------------------
          

In [10]:
#Import json handling
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [11]:
#Generate a map of the neighborhoods
# create the inital map using latitude and longitude values
to_latitude=43.66
to_longitude=-79.38
map_toronto = folium.Map(location=[to_latitude, to_longitude], zoom_start=13)

# populate map with neighborhoods
for lat, lng, borough, neighborhood in zip(df_toronto_nb['Latitude'], df_toronto_nb['Longitude'], df_toronto_nb['Borough'], df_toronto_nb['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
#print neighborhood map
map_toronto

Initilazitation of Foursquare

In [12]:
CLIENT_ID = 'G5MVYLG5OBPFMMPWB4HELV1JFQGK4UM34CE0E5TDMEML04JS' # your Foursquare ID
CLIENT_SECRET = 'SBVJK4X5PDUX43BQV2JOTTXODIQMBD2ILCLMJ5HNQAWFNC2H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G5MVYLG5OBPFMMPWB4HELV1JFQGK4UM34CE0E5TDMEML04JS
CLIENT_SECRET:SBVJK4X5PDUX43BQV2JOTTXODIQMBD2ILCLMJ5HNQAWFNC2H


Retrieve Foursquare data for all neighborhoods

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# execute above function for all neighborhoods
toronto_venues = getNearbyVenues(names=df_toronto_nb['Neighbourhood'],
                                   latitudes=df_toronto_nb['Latitude'],
                                   longitudes=df_toronto_nb['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Process the retrieved data:

In [43]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#get list
columns = list(toronto_onehot)
# move the column to head of list using index, pop and insert
columns.insert(0, columns.pop(columns.index('Neighborhood')))
toronto_onehot = toronto_onehot.loc[:, columns]


toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Characterize the neighborhoods by summarizing the venue results via their mean appearance:

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

Sort by top 5 venues:

In [45]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.03
3      Farmers Market  0.03
4         Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2       Nightclub  0.09
3     Coffee Shop  0.09
4             Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
              venue  freq
0       Yoga Studio  0.06
1     Auto Workshop  0.06
2        Comic Shop  0.06
3              Park  0.06
4  Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.14
1      Airport Service  0.14
2     Airport Terminal  0.14
3          Coffee Shop  0.07
4  Rental Car Location  0.07


----Central Bay Street----
                venue  freq
0         Coff

Define function for getting top ten venues:

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Get the top ten venues per neighborhood:

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pharmacy,BBQ Joint,Pizza Place,Pub,Middle Eastern Restaurant,Burger Joint,Donut Shop
35,The Beaches,Trail,Health Food Store,Pub,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio
36,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Pub,Pizza Place,Lounge
37,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Salad Place,Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Japanese Restaurant,Beer Bar
38,"University of Toronto, Harbord",Café,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Bookstore,Bank,Beer Bar,Beer Store,Sandwich Place


Now prepare the clustering:

In [63]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Do the clustering:

In [58]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 0, 1, 0,
       0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto_nb
toronto_merged.head()

# merge data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Breakfast Spot,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Restaurant,Café,Portuguese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Bookstore,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Clothing Store,American Restaurant,Art Gallery,Hotel,Farmers Market,Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio


In [66]:
# create map
map_clusters = folium.Map(location=[to_latitude, to_longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters